In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

caffe_root = '/home/cv/hdl/caffe'
import sys
sys.path.insert(0, caffe_root + 'python')

import caffe

caffe.set_device(0)
caffe.set_mode_gpu()

caffe_root = '/home/cv/hdl/caffe'

In [3]:
# Load the source network and extract the fully connected layers' parameters.
net = caffe.Net('{}/models/vgg16/VGG_ILSVRC_16_layers_deploy.prototxt'.format(caffe_root), 
                '{}/models/vgg16/VGG_ILSVRC_16_layers.caffemodel'.format(caffe_root), 
                caffe.TEST)
src_params = ['fc6', 'fc7']
# params = [pr for pr in net.params.keys() if 'fc8' not in pr]


In [4]:
# Load the fully convolutional network to transplant the parameters.
net_full_conv = caffe.Net('head/model/deploy.prototxt',
                          caffe.TEST)
solver = caffe.SGDSolver('head/model/solver.prototxt')
dest_params = ['fc6', 'fc7']
# params_full_conv = [pr for pr in net_full_conv.params.keys() if 'score' not in pr]


In [5]:
# First: copy shared layers
shared_layers = set(net.params.keys()) & set(net_full_conv.params.keys())
shared_layers -= set(src_params + dest_params)

In [6]:
for layer in sorted(list(shared_layers)):
    print("Copying shared layer",layer)
    net_full_conv.params[layer][0].data[...] = net.params[layer][0].data
    net_full_conv.params[layer][1].data[...] = net.params[layer][1].data

('Copying shared layer', 'conv1_1')
('Copying shared layer', 'conv1_2')
('Copying shared layer', 'conv2_1')
('Copying shared layer', 'conv2_2')
('Copying shared layer', 'conv3_1')
('Copying shared layer', 'conv3_2')
('Copying shared layer', 'conv3_3')
('Copying shared layer', 'conv4_1')
('Copying shared layer', 'conv4_2')
('Copying shared layer', 'conv4_3')
('Copying shared layer', 'conv5_1')
('Copying shared layer', 'conv5_2')
('Copying shared layer', 'conv5_3')


In [7]:
# Second: copy over fully connected layers
# fc_params = {name: (weights, biases)}
fc_params = {pr: (net.params[pr][0].data, net.params[pr][1].data) for pr in src_params}

for fc in src_params:
    print '{} weights are {} dimensional and biases are {} dimensional'.format(fc, fc_params[fc][0].shape, fc_params[fc][1].shape)

fc6 weights are (4096, 25088) dimensional and biases are (4096,) dimensional
fc7 weights are (4096, 4096) dimensional and biases are (4096,) dimensional


In [8]:
# conv_params = {name: (weights, biases)}
conv_params = {pr: (net_full_conv.params[pr][0].data, net_full_conv.params[pr][1].data) for pr in dest_params}

for conv in dest_params:
    print '{} weights are {} dimensional and biases are {} dimensional'.format(conv, conv_params[conv][0].shape, conv_params[conv][1].shape)

fc6 weights are (4096, 512, 7, 7) dimensional and biases are (4096,) dimensional
fc7 weights are (4096, 4096, 1, 1) dimensional and biases are (4096,) dimensional


In [9]:
for pr, pr_conv in zip(src_params, dest_params):
    conv_params[pr_conv][0].flat = fc_params[pr][0].flat  # flat unrolls the arrays
    conv_params[pr_conv][1][...] = fc_params[pr][1]

Next, save the new model weights.

In [10]:
net_full_conv.save('alt_vgg16fc.caffemodel')

In [15]:
print net_full_conv.params['fc7'][0].data

[[[[  3.90147837e-03]]

  [[  5.35322493e-03]]

  [[ -3.13405343e-03]]

  ..., 
  [[ -4.24027257e-03]]

  [[  2.78731284e-04]]

  [[  4.96918161e-04]]]


 [[[ -1.80806511e-03]]

  [[ -2.31539086e-03]]

  [[ -8.60718792e-05]]

  ..., 
  [[ -4.14406357e-04]]

  [[  9.56191774e-03]]

  [[  1.63843471e-03]]]


 [[[  1.36159221e-03]]

  [[ -3.61574744e-03]]

  [[  2.81324098e-03]]

  ..., 
  [[ -7.89196289e-04]]

  [[ -6.20277831e-04]]

  [[ -9.92762507e-04]]]


 ..., 
 [[[  1.05189125e-03]]

  [[ -8.41525383e-04]]

  [[  2.33084010e-03]]

  ..., 
  [[  6.79930439e-03]]

  [[ -2.16196757e-03]]

  [[ -4.49770268e-05]]]


 [[[ -5.83238620e-03]]

  [[ -1.55866274e-03]]

  [[  4.52873018e-03]]

  ..., 
  [[  3.37824528e-03]]

  [[  5.90845896e-03]]

  [[ -3.50919843e-04]]]


 [[[  1.55499880e-03]]

  [[  5.22026513e-03]]

  [[ -6.23492757e-03]]

  ..., 
  [[  2.13007792e-03]]

  [[  1.76626071e-03]]

  [[  5.03490586e-03]]]]
